In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *


#Instructions for the pipeline
Requires two inputs for training:
    - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration)
    - NetsurfP and Biopython data that has been precalculated
    - X characteristics to predict

pipeline
Take mass spec spreadsheet
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration
Merge with Proteome data to get file that has
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence
Calculate protein features using biopython
Merge with NSP data to get all protein features

Split into X and Y dataset with Entries as labels

In [4]:
### New Data workup for RFG

# Pull together Proteomic data
in_dir="Input_data/ExternalData/abundance/"
#combine Mass Spec data input into one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
#take files in_dir and combine then into one pandas df (raw_MS_data)
files= os.listdir(in_dir)
for i,f in enumerate(files):
    if i==0:
        raw_MS_data=pd.read_csv(in_dir+f,header=0)
    else:
        temp = pd.read_csv(in_dir+f,header=0)
        raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
# melt the df to make it an accession number, NPUNID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='NPUNID', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()

In [13]:
control_files=['controls_BALF.xlsx','controls_FBS.xlsx','controls_Human_unedited.xlsx']
control_dir='Input_data/Proteomic data/'
import glob
combined_list=[]
for file in control_files:
    combined_list.append(pd.read_excel(control_dir+file))
combined= pd.concat(combined_list, ignore_index=True)
combined.dropna(how='all', axis=1,inplace=True)

In [16]:
combined.to_excel(control_dir+'controls_combined.xlsx',index=False)

In [5]:
###Bring in controls (MS data for serums)##
#Controls is the abundance of the proteins in serum that have been observed via proteomics
#human protein abundance data found here- https://db.systemsbiology.net/sbeams/cgi/PeptideAtlas/GetProteins
#Normalized expected concentration into a percent to make it similar to the rest of the normalized abundances used
#Need FBS As well for Ban task
controls=combined
MS_data_controls = pd.merge(raw_MS_data,controls,how='inner', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='UniProt/Human_proteome_221201.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='NetsurfP_Proteomes/Bovine_Mouse_proteome_complete.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [4]:
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat, how='left',on='Entry')

C:\Users\kmp95\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
MS_data_clean = raw_MS_data.copy()
PROT_cleaned_data = normalize_mass_length_1DF(raw_prop_data) #function found in data_prep_functions line 167, normalizes mass, length and mw by dividing all values by the max in the column
Protein_data_complete = pd.merge(PROT_cleaned_data, NSP_data, left_on='Entry', right_on='Entry') #merges netsurfp features and biopython features
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='left', on='NPUNID')
data_complete.drop(columns=['notes','Notes','BET','NPUNID','Unnamed: 0'],inplace=True)
#Optional here to drop all enrichment values that are NA - also have to deal with positive/negative infinity values
# data_complete.dropna(subset=['Enrichment'],inplace=True)

In [11]:
data_complete.drop(columns='Unnamed: 0',inplace=True)

In [12]:
data_complete.dropna(inplace=True)
#replace -infintiy and positive infinity with -12 and 12

#####COME BACK AND FIND BETTER SOLUTION!!!##
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')

In [13]:
#set labels (what we are trying to predict) as Enrichment column
labels=data_complete['Enrichment'].copy()
#make it one dimenisional
labels=np.ravel(labels)
#drop qualitative, not neccessary, and label columns
df=data_complete.drop(['Entry','Sequence','Core Material','Ligand','NPID','Enrichment'],axis=1)

# print(df)
#these are left over metrics from the helper functions from landry paper replace with better options for looking at metrics###
first_frame = True #starting dataframe for saving metrics
correctness_frame = pd.DataFrame()
metrics_frame = pd.DataFrame()
print_metrics = 1 #0, doesn't show metrics while runnning for each model, 1 does show metrics


In [ ]:
labels

In [55]:
data_complete

,Entry,Enrichment,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,...,Core Material,Ligand,Dtem,Dh,Shaken,Centrifuged,NP_incubation Concentration (mg/mL),Incubation Concentration (mg/ml),Incubation Time (minutes),Temperature
0,P02769,-3.058069,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
1,P02769,-2.626344,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
2,P02769,-2.660705,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
6,P41361,4.586080,MISNGIGTVTAGKRSICLLPLLLIGLWGCVTCHRSPVEDVCTAKPR...,465.0,52347.0,0.064516,0.019355,0.045161,0.075269,0.053763,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
7,P41361,4.631187,MISNGIGTVTAGKRSICLLPLLLIGLWGCVTCHRSPVEDVCTAKPR...,465.0,52347.0,0.064516,0.019355,0.045161,0.075269,0.053763,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3149,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,R101,none,800.0,680.0,1.0,1.0,125.0,0.2,60.0,25.0
3150,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0
3151,Q00896,1.789023,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0
3152,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0


In [14]:
#Run recursive feature elimination to determine top features to select
#currently selecting 15 although that is arbitrary
### COME UP with better more objective way for determining number of features####
from sklearn.feature_selection import RFE

estimator = RandomForestRegressor(n_estimators=100)
selector = RFE(estimator, n_features_to_select= 15, step=1)
selector = selector.fit(df,labels)
selector.support_

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False,  True, False,  True, False, False, False, False,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
        True, False, False, False])

In [15]:
selector.ranking_

array([72, 78, 21, 39, 67, 62, 13,  8, 36,  1, 45, 23, 32, 28, 69,  1, 16,
       57, 47, 34,  1, 61,  1, 25,  4, 41, 14,  1, 11,  1,  6, 37, 65, 42,
       55,  3, 24, 84, 51, 87, 46, 86, 63, 75,  1, 80, 83,  1, 26, 12, 68,
       58, 74, 73, 64, 27, 76, 22, 59,  1, 53, 19, 48, 49,  9, 31, 54, 82,
        1,  1, 81, 38, 70, 40, 60,  1, 52, 10,  2,  5, 15, 43, 20, 56, 30,
        1, 33, 71, 18, 44,  7, 66, 50, 85,  1, 17, 29, 88, 79,  1, 35, 77,
       89])

In [16]:
feat_list=selector.get_feature_names_out()
print(feat_list)

['frac_aa_I' 'frac_aa_Q' 'frac_aa_W' 'molecular_weight' 'flexibility_var'
 'flexibility_min' 'fraction_exposed_polar_exposed' 'rsa_std'
 'fraction_total_exposed_M' 'fraction_total_exposed_Y'
 'fraction_exposed_exposed_A' 'fraction_exposed_exposed_H'
 'fraction_exposed_exposed_T' 'Zeta Potential'
 'NP_incubation Concentration (mg/mL)']


In [17]:
df_rfe=df[feat_list].copy()
df_rfe

,frac_aa_I,frac_aa_Q,frac_aa_W,molecular_weight,flexibility_var,flexibility_min,fraction_exposed_polar_exposed,rsa_std,fraction_total_exposed_M,fraction_total_exposed_Y,fraction_exposed_exposed_A,fraction_exposed_exposed_H,fraction_exposed_exposed_T,Zeta Potential,NP_incubation Concentration (mg/mL)
0,0.024712,0.032949,0.004942,0.136021,0.000695,0.936357,0.742604,0.223,0.003295,0.006590,0.082840,0.035503,0.073964,-38.0,3.2
1,0.024712,0.032949,0.004942,0.136021,0.000695,0.936357,0.742604,0.223,0.003295,0.006590,0.082840,0.035503,0.073964,-38.0,3.2
2,0.024712,0.032949,0.004942,0.136021,0.000695,0.936357,0.742604,0.223,0.003295,0.006590,0.082840,0.035503,0.073964,-38.0,3.2
3,0.024712,0.032949,0.004942,0.136021,0.000695,0.936357,0.742604,0.223,0.003295,0.006590,0.082840,0.035503,0.073964,9.0,4.0
4,0.024712,0.032949,0.004942,0.136021,0.000695,0.936357,0.742604,0.223,0.003295,0.006590,0.082840,0.035503,0.073964,100.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3149,0.048544,0.043689,0.007282,0.089950,0.000799,0.936714,0.741071,0.258,0.007282,0.002427,0.044643,0.049107,0.084821,-52.0,125.0
3150,0.048544,0.043689,0.007282,0.089950,0.000799,0.936714,0.741071,0.258,0.007282,0.002427,0.044643,0.049107,0.084821,-21.0,62.5
3151,0.048544,0.043689,0.007282,0.089950,0.000799,0.936714,0.741071,0.258,0.007282,0.002427,0.044643,0.049107,0.084821,-21.0,62.5
3152,0.048544,0.043689,0.007282,0.089950,0.000799,0.936714,0.741071,0.258,0.007282,0.002427,0.044643,0.049107,0.084821,-21.0,62.5


In [22]:

x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

0.43730337548323595


In [28]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i])

feature: importance score
frac_aa_I 0.06596630314032861
frac_aa_Q 0.05914304079572
frac_aa_W 0.06973760909851004
molecular_weight 0.055950814276992854
flexibility_var 0.060526394166567826
flexibility_min 0.06685478674919744
fraction_exposed_polar_exposed 0.06073350567070096
rsa_std 0.06858982375397228
fraction_total_exposed_M 0.07155243384020243
fraction_total_exposed_Y 0.062404576064288855
fraction_exposed_exposed_A 0.04876247693133964
fraction_exposed_exposed_H 0.05787822204456944
fraction_exposed_exposed_T 0.061567447024784984
Zeta Potential 0.09787628088636142
NP_incubation Concentration (mg/mL) 0.09245628555646303
